# Figure 5 (a), (b) & (c)


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

plt.rcParams.update({'font.size': 14})

# --- Load data ---
u10 = xr.open_dataset('/mnt/d/Data/era5_wind_2002_2023_daily.nc').u10
v10 = xr.open_dataset('/mnt/d/Data/era5_wind_2002_2023_daily.nc').v10
mslp = xr.open_dataset(
    '/mnt/d/Data/MSLP_daily_20022024.nc').msl / 100  # Convert Pa to hPa

# --- Load land-sea mask ---
lsm = xr.open_dataset(
    '/mnt/d/3b66365b36967ddc61067f1cbbb2402b.nc').lsm.isel(valid_time=0).squeeze()
ocean_mask = xr.where(lsm < 0.5, 1, np.nan)

# --- Rename time to valid_time ---
for var in [u10, v10, mslp]:
    if 'time' in var.coords:
        var = var.rename({'time': 'valid_time'})

# --- Sort latitude south to north ---
for var in [u10, v10, mslp]:
    if var.latitude[0] > var.latitude[-1]:
        var = var.sortby('latitude')

# --- Subset JJAS months ---
months = [6, 7, 8]
u_jjas = u10.sel(valid_time=u10.valid_time.dt.month.isin(months))
v_jjas = v10.sel(valid_time=v10.valid_time.dt.month.isin(months))
mslp_jjas = mslp.sel(valid_time=mslp.valid_time.dt.month.isin(months))

# --- Climatologies ---
u_clim = u_jjas.groupby('valid_time.month').mean(dim='valid_time')
v_clim = v_jjas.groupby('valid_time.month').mean(dim='valid_time')
mslp_clim = mslp_jjas.groupby('valid_time.month').mean(dim='valid_time')

# --- 2016 monthly means ---
u_2016 = u_jjas.sel(valid_time=u_jjas.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')
v_2016 = v_jjas.sel(valid_time=v_jjas.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')
mslp_2016 = mslp_jjas.sel(valid_time=mslp_jjas.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')

# --- Compute anomalies ---
anomalies = {}
for m in months:
    anomalies[m] = {
        'u': u_2016.sel(month=m) - u_clim.sel(month=m),
        'v': v_2016.sel(month=m) - v_clim.sel(month=m),
        'mslp': mslp_2016.sel(month=m) - mslp_clim.sel(month=m)
    }

# --- Plotting ---
fig, axs = plt.subplots(1, 3, figsize=(20, 8), subplot_kw={
                        'projection': ccrs.PlateCarree()})
month_names = {6: 'June', 7: 'July', 8: 'August'}

cs = None  # for shared colorbar
for i, (ax, month) in enumerate(zip(axs, months)):
    data = anomalies[month]
    u = data['u']
    v = data['v']
    mslp_anom = data['mslp'] * ocean_mask
    ax.set_extent([0, 80, -70, -50], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.set_aspect(3)
    ax.add_feature(cfeature.LAND, color='lightgray', zorder=0)

    # --- Filled MSLP anomaly contour ---
    levels = np.linspace(-12, 12, 50)
    cs = ax.contourf(mslp_anom.longitude, mslp_anom.latitude, mslp_anom,
                     levels=levels, cmap='RdBu_r', extend='both', zorder=1)

    # --- Wind vectors ---
    skip_lon = slice(None, None, 12)
    skip_lat = slice(None, None, 4)
    ax.quiver(u.longitude[skip_lon], u.latitude[skip_lat],
              (u * ocean_mask).values[skip_lat, skip_lon],
              (v * ocean_mask).values[skip_lat, skip_lon],
              scale=80, color='black', zorder=3)

    ds = xr.open_dataset(
        '/mnt/d/Data/sea_ice_20022024_reprojected.nc').sel(x=slice(36, 60), y=slice(-63, -66))
    sea_ice_concentration = ds['z']

    # Calculate mean sea ice concentration for August 2016
    sea_ice_seasonal = sea_ice_concentration.sel(
        time=slice('2016-08-11', '2016-08-12')).mean(dim='time')

    # Plot contour lines for SIC (only 30% and optionally others)
    contour = ax.contour(sea_ice_seasonal.x, sea_ice_seasonal.y, sea_ice_seasonal,
                         levels=[30], linewidths=1.5, colors='violet')

    # --- Gridlines ---
    gl = ax.gridlines(draw_labels=True, linewidth=0.5,
                      color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.bottom_labels = True
    gl.left_labels = (i == 0)

# --- Shared colorbar ---
cbar = fig.colorbar(cs, ax=axs.tolist(), orientation='horizontal',
                    pad=0.1, aspect=50, ticks=np.arange(-12, 14, 2))
cbar.set_label('Mean Sea Level Pressure Anomaly (hPa)')

plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/mslp_anom.png',
            bbox_inches='tight', dpi=300)

# Figure 5 (d), (e) & (f)


In [ ]:
import xarray as xr
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import numpy as np

# --- Set global font size ---
plt.rcParams.update({'font.size': 14})

# --- Load data ---
IWV = xr.open_mfdataset('/mnt/d/Data/IWV_daily_20022024.nc').tcwv
u10 = xr.open_dataset('/mnt/d/Data/era5_wind_2002_2023_daily.nc').u10
v10 = xr.open_dataset('/mnt/d/Data/era5_wind_2002_2023_daily.nc').v10
mslp = xr.open_dataset(
    '/mnt/d/Data/MSLP_daily_20022024.nc').msl / 100  # Convert Pa to hPa

# --- Load land-sea mask and use first timestep ---
lsm = xr.open_dataset(
    '/mnt/d/3b66365b36967ddc61067f1cbbb2402b.nc').lsm.isel(valid_time=0).squeeze()

# --- Rename time to valid_time ---
for var in [IWV, u10, v10, mslp]:
    if 'time' in var.coords:
        var = var.rename({'time': 'valid_time'})

# --- Sort latitude south to north ---
for var in [IWV, u10, v10, mslp]:
    if var.latitude[0] > var.latitude[-1]:
        var = var.sortby('latitude')

# --- Subset JJA months ---
months = [6, 7, 8]
iwv_jjas = IWV.sel(valid_time=IWV.valid_time.dt.month.isin(months))
u_jjas = u10.sel(valid_time=u10.valid_time.dt.month.isin(months))
v_jjas = v10.sel(valid_time=v10.valid_time.dt.month.isin(months))
mslp_jjas = mslp.sel(valid_time=mslp.valid_time.dt.month.isin(months))

# --- Climatologies ---
iwv_clim = iwv_jjas.groupby('valid_time.month').mean(dim='valid_time')
u_clim = u_jjas.groupby('valid_time.month').mean(dim='valid_time')
v_clim = v_jjas.groupby('valid_time.month').mean(dim='valid_time')
mslp_clim = mslp_jjas.groupby('valid_time.month').mean(dim='valid_time')

# --- 2016 Monthly Means ---
iwv_2016 = iwv_jjas.sel(valid_time=iwv_jjas.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')
u_2016 = u_jjas.sel(valid_time=u_jjas.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')
v_2016 = v_jjas.sel(valid_time=v_jjas.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')
mslp_2016 = mslp_jjas.sel(valid_time=mslp_jjas.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')

# --- Interpolate LSM to match variable grid ---
# Only need one since grid is same
lsm_interp = lsm.interp_like(iwv_2016.sel(month=6), method="nearest")

# --- Compute Anomalies ---
anomalies = {}
for m in months:
    anomalies[m] = {
        'iwv': (iwv_2016.sel(month=m) - iwv_clim.sel(month=m)).where(lsm_interp == 0),
        'u': (u_2016.sel(month=m) - u_clim.sel(month=m)).where(lsm_interp == 0),
        'v': (v_2016.sel(month=m) - v_clim.sel(month=m)).where(lsm_interp == 0),
        'mslp': (mslp_2016.sel(month=m) - mslp_clim.sel(month=m)).where(lsm_interp == 0)
    }

# --- Plotting ---
fig, axes = plt.subplots(1, 3, figsize=(20, 6), subplot_kw={
                         'projection': ccrs.PlateCarree()})
month_names = {6: 'June', 7: 'July', 8: 'August'}

levels = np.linspace(-2, 2, 21)
cmap = 'RdBu_r'

for i, (ax, month) in enumerate(zip(axes, months)):
    data = anomalies[month]
    iwv_anom = data['iwv']
    u = data['u']
    v = data['v']
    mslp_month = data['mslp']

    ax.set_extent([0, 80, -70, -50], crs=ccrs.PlateCarree())
    ax.coastlines(zorder=5)
    ax.set_aspect(3)

    # --- Add land feature ---
    ax.add_feature(cfeature.LAND, color='lightgray', zorder=1)

    # --- IWV Anomaly Contourf ---
    cf = ax.contourf(iwv_anom.longitude, iwv_anom.latitude, iwv_anom,
                     levels=levels, cmap=cmap, extend='both',
                     transform=ccrs.PlateCarree(), zorder=2)

    # --- Wind Vectors ---
    skip_lon = slice(None, None, 12)
    skip_lat = slice(None, None, 4)
    u_sub = u.isel(latitude=skip_lat, longitude=skip_lon)
    v_sub = v.isel(latitude=skip_lat, longitude=skip_lon)
    ax.quiver(u_sub.longitude, u_sub.latitude, u_sub, v_sub,
              scale=80, color='black', zorder=3)

    ds = xr.open_dataset(
        '/mnt/d/Data/sea_ice_20022024_reprojected.nc').sel(x=slice(36, 60), y=slice(-63, -66))
    sea_ice_concentration = ds['z']

    # Calculate mean sea ice concentration for August 2016
    sea_ice_seasonal = sea_ice_concentration.sel(
        time=slice('2016-08-11', '2016-08-12')).mean(dim='time')

    # Plot contour lines for SIC (only 30% and optionally others)
    contour = ax.contour(sea_ice_seasonal.x, sea_ice_seasonal.y, sea_ice_seasonal,
                         levels=[30], linewidths=1.5, colors='violet')

    # --- Gridlines ---
    gl = ax.gridlines(draw_labels=True, linewidth=0.5,
                      color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False

    if i > 0:
        gl.left_labels = False

# --- Shared Larger Colorbar ---

cbar = fig.colorbar(cf, ax=axes.tolist(), orientation='horizontal',
                    pad=0.1, aspect=50, ticks=np.arange(-2, 2.5, 0.5))
cbar.set_label('Integrated Water Vapour Anomaly (kg/m²)')
plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/IWV_anom.png',
            bbox_inches='tight', dpi=300)

plt.show()

# Figure 5 (g), (h) & (i)


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import cartopy.crs as ccrs
import cartopy.feature as cfeature
import matplotlib.patches as mpatches

plt.rcParams.update({'font.size': 14})

# --- Load heat flux components ---
latent = xr.open_dataset('/mnt/d/Data/Latent_daily_20022024.nc').avg_slhtf
sensible = xr.open_dataset('/mnt/d/Data/Sensible_daily_20022024.nc').avg_ishf
shortwave = xr.open_dataset(
    '/mnt/d/Data/Shortwave_daily_20022024.nc').avg_snswrf
longwave = xr.open_dataset('/mnt/d/Data/Longwave_daily_20022024.nc').avg_snlwrf

# --- Load wind and MSLP ---
u10 = xr.open_dataset('/mnt/d/Data/era5_wind_2002_2023_daily.nc').u10
v10 = xr.open_dataset('/mnt/d/Data/era5_wind_2002_2023_daily.nc').v10
mslp = xr.open_dataset(
    '/mnt/d/Data/MSLP_daily_20022024.nc').msl / 100  # Pa to hPa

# --- Load land-sea mask and generate ocean mask ---
lsm = xr.open_dataset(
    '/mnt/d/3b66365b36967ddc61067f1cbbb2402b.nc').lsm.isel(valid_time=0).squeeze()
ocean_mask = xr.where(lsm < 0.5, 1, np.nan)

# --- Interpolate ocean mask to flux grid for alignment ---
ocean_mask = ocean_mask.interp(
    latitude=latent.latitude, longitude=latent.longitude)

# --- Align time across datasets ---
time_index = latent.valid_time
sensible = sensible.reindex(valid_time=time_index, method="nearest")
shortwave = shortwave.reindex(valid_time=time_index, method="nearest")
longwave = longwave.reindex(valid_time=time_index, method="nearest")

# --- Calculate net surface heat flux ---
net_flux = latent + sensible + shortwave + longwave  # Units: W/m²

# --- Ensure latitude is increasing (south to north) ---
for var in [net_flux, u10, v10, mslp, ocean_mask]:
    if var.latitude[0] > var.latitude[-1]:
        var = var.sortby('latitude')

# --- JJA months ---
months = [6, 7, 8]

# --- Subset only by months ---
hf_jja = net_flux.sel(valid_time=net_flux.valid_time.dt.month.isin(months))
u_jja = u10.sel(valid_time=u10.valid_time.dt.month.isin(months))
v_jja = v10.sel(valid_time=v10.valid_time.dt.month.isin(months))
mslp_jja = mslp.sel(valid_time=mslp.valid_time.dt.month.isin(months))

# --- Monthly climatologies ---
hf_clim = hf_jja.groupby('valid_time.month').mean(dim='valid_time')
u_clim = u_jja.groupby('valid_time.month').mean(dim='valid_time')
v_clim = v_jja.groupby('valid_time.month').mean(dim='valid_time')
mslp_clim = mslp_jja.groupby('valid_time.month').mean(dim='valid_time')

# --- 2016 monthly means ---
hf_2016 = hf_jja.sel(valid_time=hf_jja.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')
u_2016 = u_jja.sel(valid_time=u_jja.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')
v_2016 = v_jja.sel(valid_time=v_jja.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')
mslp_2016 = mslp_jja.sel(valid_time=mslp_jja.valid_time.dt.year == 2016).groupby(
    'valid_time.month').mean(dim='valid_time')

# --- Compute anomalies and apply ocean mask ---
anomalies = {}
for m in months:
    anomalies[m] = {
        'net_flux': (hf_2016.sel(month=m) - hf_clim.sel(month=m)) * ocean_mask,
        'u': (u_2016.sel(month=m) - u_clim.sel(month=m)) * ocean_mask,
        'v': (v_2016.sel(month=m) - v_clim.sel(month=m)) * ocean_mask,
        # MSLP not masked
        'mslp': mslp_2016.sel(month=m) - mslp_clim.sel(month=m)
    }

# --- Plotting ---
fig, axs = plt.subplots(1, 3, figsize=(20, 8), subplot_kw={
                        'projection': ccrs.PlateCarree()})
month_names = {6: 'June', 7: 'July', 8: 'August'}
cmap = 'RdBu_r'
levels = np.linspace(-80, 80, 21)

for i, month in enumerate(months):
    ax = axs[i]
    data = anomalies[month]
    flux_anom = data['net_flux']
    u = data['u']
    v = data['v']
    mslp_month = data['mslp']

    ax.set_extent([0, 80, -70, -50], crs=ccrs.PlateCarree())
    ax.coastlines()
    ax.add_feature(cfeature.LAND, color='lightgray')
    ax.set_aspect(3)

    # --- Contourf of heat flux anomaly (masked) ---
    cs = ax.contourf(flux_anom.longitude, flux_anom.latitude, flux_anom,
                     levels=levels, cmap=cmap, extend='both')

    # --- Wind Vectors (subsampled and masked) ---
    skip_lon = slice(None, None, 12)
    skip_lat = slice(None, None, 4)
    u_plot = u.values[skip_lat, skip_lon]
    v_plot = v.values[skip_lat, skip_lon]

    ax.quiver(u.longitude[skip_lon], u.latitude[skip_lat],
              u_plot, v_plot, scale=80, color='black', zorder=3)
    # --- Add Box ---
    rect = mpatches.Rectangle(
        (30, -67),  # lower-left corner (lon, lat)
        30,         # width in degrees (60-20)
        7,         # height in degrees (-60 - (-70))
        linewidth=2,
        edgecolor='green',
        facecolor='none',
        transform=ccrs.PlateCarree(),
        zorder=4
    )
    ax.add_patch(rect)
    ds = xr.open_dataset(
        '/mnt/d/Data/sea_ice_20022024_reprojected.nc').sel(x=slice(36, 60), y=slice(-63, -66))
    sea_ice_concentration = ds['z']  # Adjust variable name as needed

    # Calculate mean sea ice concentration for August 2016
    sea_ice_seasonal = sea_ice_concentration.sel(
        time=slice('2016-08-11', '2016-08-12')).mean(dim='time')

    # Plot contour lines for SIC (only 30% and optionally others)
    contour = ax.contour(sea_ice_seasonal.x, sea_ice_seasonal.y, sea_ice_seasonal,
                         levels=[30], linewidths=1.5, colors='violet')

    # --- Gridlines ---
    gl = ax.gridlines(draw_labels=True, linewidth=0.5,
                      color='gray', alpha=0.5, linestyle='--')
    gl.top_labels = False
    gl.right_labels = False
    gl.left_labels = (i == 0)

# --- Shared colorbar ---
cbar = fig.colorbar(cs, ax=axs[:3].tolist(
), orientation='horizontal', pad=0.1, aspect=50, ticks=np.arange(-80, 100, 20))
cbar.set_label('Net Atmospheric Heat Flux Anomaly (W/m²)')
plt.savefig('/home/soumya/Backup/Plots/Cosmonaut Polynya/nhf_anom.png',
            bbox_inches='tight', dpi=300)
plt.show()

# Figure 5 (j)


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load datasets
latent = xr.open_dataset(
    '/mnt/d/Data/latent_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).mslhf
sensible = xr.open_dataset(
    '/mnt/d/Data/sensible_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msshf
shortwave = xr.open_dataset(
    '/mnt/d/Data/shortwave_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msnswrf
longwave = xr.open_dataset(
    '/mnt/d/Data/longwave_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msnlwrf

# Ensure time alignment by reindexing time (if necessary)
time_index = latent.valid_time
sensible = sensible.reindex(valid_time=time_index, method="nearest")
shortwave = shortwave.reindex(valid_time=time_index, method="nearest")
longwave = longwave.reindex(valid_time=time_index, method="nearest")

# Calculate the spatial mean for each heat flux component
latent_mean = latent.mean(dim=['latitude', 'longitude'])
sensible_mean = sensible.mean(dim=['latitude', 'longitude'])
shortwave_mean = shortwave.mean(dim=['latitude', 'longitude'])
longwave_mean = longwave.mean(dim=['latitude', 'longitude'])

# Calculate net heat flux: positive values indicate heat input to the ocean
net_heat_flux = shortwave_mean + longwave_mean + sensible_mean + latent_mean

# Calculate monthly climatology (2002–2022) for net heat flux
net_climatology = net_heat_flux.sel(valid_time=slice(
    '2002', '2022')).groupby('valid_time.month').mean()

# Calculate anomalies for the entire dataset
net_anomalies = net_heat_flux.groupby('valid_time.month') - net_climatology

# Apply a 90-day moving average to the net heat flux anomalies
net_anomalies_ma = net_anomalies.rolling(
    valid_time=90, min_periods=1, center=False).mean()

# Plotting the net heat flux anomaly for the entire dataset
plt.figure(figsize=(18, 6))

# Plot net heat flux anomalies
plt.plot(
    net_anomalies_ma.valid_time, net_anomalies_ma,
    label='Net Heat Flux Anomalies (90-day MA)', color='red', linewidth=2
)

# Add zero line for reference
plt.axhline(0, color='black', linestyle='--', linewidth=1)

# Customize the plot
plt.ylabel('Net Heat Flux Anomaly (W/m²)', fontsize=15)
plt.grid(True, linestyle='--', alpha=0.6)

# Set x-axis limits to remove empty spaces
plt.xlim(net_anomalies_ma.valid_time[0].values,
         net_anomalies_ma.valid_time[-1].values)

# Format x-ticks to show years
years = pd.date_range(start="2002-01-01", end="2023-12-31", freq="YS")
plt.xticks(years, [date.strftime("%Y") for date in years],
           rotation=45, fontsize=14)  # Increased font size
plt.yticks(fontsize=14)  # Increased font size for y-axis ticks

# Save the plot
plt.savefig(
    '/home/soumya/Backup/Plots/Net_Heat_Flux_Anomalies_2002_2023_90dMA.png', dpi=300)
plt.show()

# Figure S4


In [ ]:
import xarray as xr
import numpy as np
import matplotlib.pyplot as plt
import pandas as pd

# Load datasets
latent = xr.open_dataset(
    '/mnt/d/Data/latent_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).mslhf
sensible = xr.open_dataset(
    '/mnt/d/Data/sensible_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msshf
shortwave = xr.open_dataset(
    '/mnt/d/Data/shortwave_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msnswrf
longwave = xr.open_dataset(
    '/mnt/d/Data/longwave_heat_2002_2023.nc').sel(latitude=slice(-62, -67)).msnlwrf

# Ensure time alignment by reindexing time (if necessary)
time_index = latent.valid_time
sensible = sensible.reindex(valid_time=time_index, method="nearest")
shortwave = shortwave.reindex(valid_time=time_index, method="nearest")
longwave = longwave.reindex(valid_time=time_index, method="nearest")

# Calculate the spatial mean for each heat flux component
latent_mean = latent.mean(dim=['latitude', 'longitude'])
sensible_mean = sensible.mean(dim=['latitude', 'longitude'])
shortwave_mean = shortwave.mean(dim=['latitude', 'longitude'])
longwave_mean = longwave.mean(dim=['latitude', 'longitude'])

# Calculate climatologies for each component (2002–2022)
latent_climatology = latent_mean.sel(valid_time=slice(
    '2002', '2022')).groupby('valid_time.month').mean()
sensible_climatology = sensible_mean.sel(valid_time=slice(
    '2002', '2022')).groupby('valid_time.month').mean()
shortwave_climatology = shortwave_mean.sel(valid_time=slice(
    '2002', '2022')).groupby('valid_time.month').mean()
longwave_climatology = longwave_mean.sel(valid_time=slice(
    '2002', '2022')).groupby('valid_time.month').mean()

# Calculate anomalies for each component
latent_anomalies = latent_mean.groupby('valid_time.month') - latent_climatology
sensible_anomalies = sensible_mean.groupby(
    'valid_time.month') - sensible_climatology
shortwave_anomalies = shortwave_mean.groupby(
    'valid_time.month') - shortwave_climatology
longwave_anomalies = longwave_mean.groupby(
    'valid_time.month') - longwave_climatology

# Apply a 90-day moving average to each component
latent_anomalies_ma = latent_anomalies.rolling(
    valid_time=90, min_periods=1, center=False).mean()
sensible_anomalies_ma = sensible_anomalies.rolling(
    valid_time=90, min_periods=1, center=False).mean()
shortwave_anomalies_ma = shortwave_anomalies.rolling(
    valid_time=90, min_periods=1, center=False).mean()
longwave_anomalies_ma = longwave_anomalies.rolling(
    valid_time=90, min_periods=1, center=False).mean()

# Plot each anomaly in a separate subplot
fig, axes = plt.subplots(4, 1, figsize=(16, 14), sharex=True)

# Plot latent heat flux anomalies
axes[0].plot(latent_anomalies_ma.valid_time, latent_anomalies_ma, color='blue')
axes[0].axhline(0, color='black', linestyle='--', linewidth=1)
axes[0].set_ylabel('Latent Heat Flux (W/m²)', fontsize=14)
axes[0].grid(True, linestyle='--', alpha=0.6)

# Plot sensible heat flux anomalies
axes[1].plot(sensible_anomalies_ma.valid_time,
             sensible_anomalies_ma, color='orange')
axes[1].axhline(0, color='black', linestyle='--', linewidth=1)
axes[1].set_ylabel('Sensible Heat Flux (W/m²)', fontsize=14)
axes[1].grid(True, linestyle='--', alpha=0.6)

# Plot shortwave heat flux anomalies
axes[2].plot(shortwave_anomalies_ma.valid_time,
             shortwave_anomalies_ma, color='green')
axes[2].axhline(0, color='black', linestyle='--', linewidth=1)
axes[2].set_ylabel('Shortwave Heat Flux (W/m²)', fontsize=14)
axes[2].grid(True, linestyle='--', alpha=0.6)

# Plot longwave heat flux anomalies
axes[3].plot(longwave_anomalies_ma.valid_time,
             longwave_anomalies_ma, color='red')
axes[3].axhline(0, color='black', linestyle='--', linewidth=1)
axes[3].set_ylabel('Longwave Heat Flux (W/m²)', fontsize=14)
axes[3].grid(True, linestyle='--', alpha=0.6)

# Customize x-axis for the bottom subplot
axes[3].set_xlabel('Time', fontsize=16)

# Add x-ticks for all years
years = pd.date_range(start="2002-01-01", end="2023-12-31", freq="YS")
axes[3].set_xticks(years)
axes[3].set_xticklabels([date.strftime("%Y")
                        for date in years], rotation=45, fontsize=14)

# Set x-axis limits to remove extra spaces
start_time = pd.Timestamp("2002-01-01")
end_time = pd.Timestamp("2023-12-31")
for ax in axes:
    ax.set_xlim([start_time, end_time])
    ax.tick_params(axis='y', labelsize=12)


# Adjust layout and save the plot
plt.tight_layout()
plt.savefig('/home/soumya/Backup/Plots/Heat_Flux_Anomalies_Components_2002_2023_90dMA.png',
            dpi=300, bbox_inches='tight')
plt.show()

# Figure S5


In [ ]:
import xarray as xr
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.dates as mdates

# Load wind stress data
u = xr.open_mfdataset('/mnt/d/Data/u10.nc').sel(valid_time=slice('2016-01-01',
                                                                 '2016-12-31'), longitude=slice(30, 60), latitude=slice(-60, -67)).u10
v = xr.open_mfdataset('/mnt/d/Data/v10.nc').sel(valid_time=slice('2016-01-01',
                                                                 '2016-12-31'), longitude=slice(30, 60), latitude=slice(-60, -67)).v10

# Calculate wind stress components
wd = (u**2 + v**2) ** 0.5
cd = 1.3 * (10**-3)
tau_x = 1.3 * cd * u * wd
tau_y = 1.3 * cd * v * wd

# Calculate grid spacing
dlat = u.latitude.diff('latitude')
dlon = u.longitude.diff('longitude')
dy_m = np.abs(111320 * dlat)
dx_m = 111320 * dlon * np.cos(np.deg2rad(u.latitude))

# Calculate wind stress curl
d_tau_y_dx = tau_y.differentiate('longitude') / dx_m
d_tau_x_dy = tau_x.differentiate('latitude') / dy_m
wind_stress_curl = d_tau_y_dx - d_tau_x_dy
curl = wind_stress_curl.mean(dim=('longitude', 'latitude'))

# Filter curl for JJAS
curl_jjas = curl.sel(valid_time=curl['valid_time'].dt.month.isin([6, 7, 8, 9]))

# Load polynya area data
polynya_df = pd.read_csv('/mnt/d/Data/Polynya Area/AMSR/Polynya_Area_30.csv')
polynya_df['Timestamp'] = pd.to_datetime(polynya_df['Timestamp'])

# Filter data for 2023 and JJAS months
polynya_df = polynya_df[(polynya_df['Timestamp'].dt.year == 2016) &
                        (polynya_df['Timestamp'].dt.month.isin([6, 7, 8, 9]))]
polynya_df = polynya_df[(polynya_df['lat'] >= -67) & (polynya_df['lat'] <= -60) &
                        (polynya_df['lon'] >= 30) & (polynya_df['lon'] <= 60)]
polynya_area = polynya_df.groupby(
    'Timestamp')['total_area'].sum().reset_index()
polynya_area['total_area'] = polynya_area['total_area'] / \
    (10**9)  # Convert to square kilometers

# Plotting
fig, ax1 = plt.subplots(figsize=(20, 10))

# Plot wind stress curl on primary y-axis
ax1.plot(curl_jjas['valid_time'], curl_jjas * 10**5, color='tab:blue',
         linewidth=2, linestyle='-', label='Wind Stress Curl (10$^{-5}$ N m$^{-3}$)')
# ax1.invert_yaxis()

# Plot polynya area on secondary y-axis
ax2 = ax1.twinx()
ax2.plot(polynya_area['Timestamp'], polynya_area['total_area'], color='tab:orange',
         linewidth=2, linestyle='-', label='Polynya Area (x $10^{3}$ km²)')

# Format the x-axis for full dates
ax1.xaxis.set_major_locator(mdates.MonthLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%b'))
# fig.autofmt_xdate()  # Rotate date labels

# Add grid, labels, and title
ax1.grid(True, linestyle='--', alpha=0.7)
# ax1.set_title('Wind Stress Curl and Polynya Area (JJAS 2023)', fontsize=24, weight='bold')  # Increased font size
# ax1.set_xlabel('Time (Months)', fontsize=20)  # Increased font size
ax1.set_ylabel('Wind Stress Curl (10$^{-5}$ N/m$^{3}$)',
               fontsize=16, color='tab:blue')  # Increased font size
# Increased font size
ax2.set_ylabel('Polynya Area (x $10^{3}$ km²)',
               fontsize=16, color='tab:orange')

# Enhance readability
ax1.tick_params(axis='both', which='major', labelsize=16)  # Larger tick labels
ax2.tick_params(axis='y', labelsize=16)  # Larger tick labels

plt.tight_layout()
# Save and show the plot
plt.savefig(
    '/home/soumya/Backup/Plots/Cosmonaut Polynya/wind_curl_polynya_area_JJAS_2016_large_font.png', dpi=400)
plt.show()

# Figure S6


In [ ]:
import xarray as xr
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import matplotlib.dates as mdates
from scipy.stats import pearsonr

plt.rcParams.update({'font.size': 14})

# --- Load and process wind data ---
u = xr.open_mfdataset(
    '/mnt/d/Data/u10.nc').sel(longitude=slice(30, 60), latitude=slice(-60, -67)).u10
v = xr.open_mfdataset(
    '/mnt/d/Data/v10.nc').sel(longitude=slice(30, 60), latitude=slice(-60, -67)).v10

# Compute wind speed and stress
wd = (u**2 + v**2) ** 0.5
cd = 1.3e-3
tau_x = 1.3 * cd * u * wd
tau_y = 1.3 * cd * v * wd

# Grid spacing
lat2d, lon2d = np.meshgrid(u.latitude, u.longitude, indexing='ij')
dx_m = 111320 * np.cos(np.deg2rad(lat2d)) * \
    u.longitude.diff('longitude').mean().item()
dy_m = 111320 * u.latitude.diff('latitude').mean().item()

# Compute wind stress curl
d_tau_y_dx = tau_y.differentiate('longitude') / dx_m
d_tau_x_dy = tau_x.differentiate('latitude') / dy_m
wind_stress_curl = (d_tau_y_dx - d_tau_x_dy) / 1e-7

# Area average
wind_curl_avg = wind_stress_curl.mean(dim=('longitude', 'latitude'))
wind_curl_avg.name = 'wind_stress_curl'

# Rolling mean
wind_curl_rolling = wind_curl_avg.rolling(valid_time=365, center=True).mean()

# --- Load and process MSLP ---
dr = xr.open_dataset('/mnt/d/Data/MSLP_daily_20022024.nc')
mslp = dr.msl.sel(longitude=slice(30, 60),
                  latitude=slice(-60, -67)) / 100.0  # Pa to hPa

# Area average and rolling mean
mslp_avg = mslp.mean(dim=('longitude', 'latitude'))
mslp_avg.name = 'mslp'
mslp_rolling = mslp_avg.rolling(valid_time=365, center=True).mean()

# --- Load SAM index data ---
sam_df = pd.read_csv('/mnt/d/Data/SAM.txt', delim_whitespace=True, index_col=0)
sam_df.columns = ['JAN', 'FEB', 'MAR', 'APR', 'MAY', 'JUN',
                  'JUL', 'AUG', 'SEP', 'OCT', 'NOV', 'DEC']

# Reshape to long format with datetime index
sam_long = sam_df.stack().reset_index()
sam_long.columns = ['Year', 'Month', 'SAM']
sam_long['Month'] = pd.to_datetime(sam_long['Month'], format='%b').dt.month
sam_long['Date'] = pd.to_datetime(sam_long[['Year', 'Month']].assign(DAY=15))
sam_long = sam_long.set_index('Date').sort_index()

# Rolling mean for SAM
sam_rolling = sam_long['SAM'].rolling(12, center=True).mean()

# --- Align and merge data ---
mslp_df = mslp_rolling.to_dataframe().dropna().reset_index().set_index('valid_time')
curl_df = wind_curl_rolling.to_dataframe(
).dropna().reset_index().set_index('valid_time')
sam_df_aligned = sam_rolling.to_frame().dropna()

# Combine all
df_all = pd.concat([sam_df_aligned, mslp_df, curl_df], axis=1).dropna()

# --- Correlation analysis ---
r_sam_mslp, p_sam_mslp = pearsonr(df_all['SAM'], df_all['mslp'])
r_sam_curl, p_sam_curl = pearsonr(df_all['SAM'], df_all['wind_stress_curl'])

# --- Plot all time series ---
fig, ax1 = plt.subplots(figsize=(18, 8))

# SAM Index (primary y-axis)
ax1.plot(df_all.index, df_all['SAM'], label='SAM Index', color='black')
ax1.set_ylabel('SAM Index', color='black')
ax1.tick_params(axis='y', labelcolor='black')

# Format x-axis to show all years
ax1.xaxis.set_major_locator(mdates.YearLocator())
ax1.xaxis.set_major_formatter(mdates.DateFormatter('%Y'))
plt.setp(ax1.xaxis.get_majorticklabels(), rotation=45)

# MSLP (second y-axis)
ax2 = ax1.twinx()
ax2.plot(df_all.index, df_all['mslp'], label='MSLP (hPa)', color='tab:blue')
ax2.set_ylabel('MSLP (hPa)', color='tab:blue')
ax2.tick_params(axis='y', labelcolor='tab:blue')
ax2.invert_yaxis()

# Wind Stress Curl (third y-axis)
ax3 = ax1.twinx()
ax3.spines['right'].set_position(('axes', 1.05))
ax3.plot(df_all.index, df_all['wind_stress_curl'],
         label='Wind Stress Curl', color='tab:red')
ax3.set_ylabel('Wind Stress Curl (x10$^{-7}$ N m$^{-3}$)', color='tab:red')
ax3.tick_params(axis='y', labelcolor='tab:red')

# Title and formatting
fig.tight_layout()
plt.grid(True, linestyle='--', alpha=0.3)

plt.savefig(
    '/home/soumya/Backup/Plots/Cosmonaut Polynya/mslp_wind_corr.png', dpi=300)
# --- Print correlation results ---
print(f"Correlation (SAM vs MSLP): r = {r_sam_mslp:.2f}, p = {p_sam_mslp:.3f}")
print(
    f"Correlation (SAM vs Wind Stress Curl): r = {r_sam_curl:.2f}, p = {p_sam_curl:.4f}")